In [2]:
import pandas as pd
import json
import re
import os
import numpy as np
#from tinydb import TinyDB
#import requests
#from bs4 import BeautifulSoup
#from urllib.request import urlopen

# NLP
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords

workDir = '/home/lefko/git/ut-health-project'
dbfile = workDir + '/db/tweets.csv'

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [3]:
# check if the JSON file can be accessed
try:
    f = open(dbfile)
except IOError:
    print("File not accessible")
finally:
    f.close()


In [4]:
"""
@Deprecated
def getCitiesPerState(state):
    '''
        @param state must begin with an uppercase letter
        scrape the list of cities and towns in the respective state
    '''
    raise DeprecationWarning('Not working, sadly.')
    cities = []
    url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_' + state
    print('url to scrape:', url)
    website = requests.get(url)
    # load the retrieved site with BS
    soup = BeautifulSoup(website.text,'html.parser')
    
    # cities/towns are stored in an element with class 'wikitable sortable'
    cities_table = soup.find_all("table", class_ = "wikitable sortable")
    print(len(cities_table))
    '''
    cities_links = cities_table.findAll('a') # city names are links in the table
    for link in cities_links:
        cities.append(link.get('title'))
    '''
    return cities
"""
def readCitiesFromFile(state):
    cities = []
    with open(workDir + '/data/cities_' + state + '.txt') as txt:
        for line in txt:
            cities.append(line.strip().lower())

    return pd.DataFrame(cities, columns = ['city'])

def readKeywords():
    keywords = []

    with open(workDir + '/data/keywords.txt') as txt:
        for line in txt:
            keywords.append(line.strip().lower())

    print(keywords)
    return keywords


In [5]:
cities_colorado = readCitiesFromFile('co')
cities_california = readCitiesFromFile('ca')
cities_massachussetts = readCitiesFromFile('ma')
cities_all = cities_california + cities_colorado + cities_massachussetts
print(cities_colorado[0:10])
print(cities_california[0:10])
print(cities_massachussetts[0:10])

city
0        acres green
1            aguilar
2  air force academy
3              akron
4            alamosa
5       alamosa east
6         allenspark
7               alma
8           antonito
9          applewood
             city
0  acalanes ridge
1          acampo
2           acton
3        adelanto
4            adin
5    agoura hills
6      agua dulce
7         aguanga
8        ahwahnee
9         airport
              city
0         abington
1  acushnet center
2            adams
3      agawam town
4    amesbury town
5   amherst center
6          andover
7        arlington
8            athol
9        attleboro


In [6]:
tweets = pd.read_csv(dbfile, names=['tweet_id', 'created', 'loc', 'text'], header=None)
tweets = tweets[tweets['loc'].isnull() == False] # sort out NaN places
tweets = tweets[tweets['loc'].str.contains('\d') == False]
tweets = tweets.applymap(lambda s:s.lower() if type(s) == str else s)
print(len(tweets), 'tweets loaded!') # tweets having a location

20189 tweets loaded!


In [7]:
print(tweets.head(15))
print(tweets.tail(15))

tweet_id              created                       loc  \
0   1211029295977492481  2019-12-28t21:00:58       greater hartford ct   
1   1211029297894326272  2019-12-28t21:00:59             providence ri   
2   1211029301195358209  2019-12-28t21:00:59                  new york   
3   1211029301791002624  2019-12-28t21:01:00           connecticut usa   
5   1211029330740043777  2019-12-28t21:01:06          rhode island usa   
6   1211029331738349569  2019-12-28t21:01:07                 boston ma   
7   1211029338596028418  2019-12-28t21:01:08              waterbury ct   
9   1211029342719004674  2019-12-28t21:01:09             charleston sc   
10  1211029350621036545  2019-12-28t21:01:11                 lowell ma   
11  1211029350910451712  2019-12-28t21:01:11          tampa fl she her   
12  1211029356174282752  2019-12-28t21:01:13  dorchester massachusetts   
14  1211029376206344193  2019-12-28t21:01:17            virginia beach   
15  1211029377414303745  2019-12-28t21:01:18         

In [8]:
# How unique are the users locations?
print(tweets['loc'].unique())
#print(tweets['loc'].unique().tolist())

['greater hartford ct' 'providence ri' 'new york' ... 'corona del mar'
 'losa ccusa' 'por todo l a califas']


In [9]:
# check for some real location names
tweets[tweets['loc'].isin(cities_all) == True]
# this approach is not working, due to the reason, that the city names in the location field are not even similar to the actual ones
# e.g. due to misspellings or out of secrecy

,tweet_id,created,loc,text


In [10]:
# These tweets contain locations, either real or fake
tweets[tweets['loc'].str.contains('|'.join([' ma', ' co', ' ca']))]

,tweet_id,created,loc,text
6,1211029331738349569,2019-12-28t21:01:07,boston ma,this industrial loft is for sale go to https t...
10,1211029350621036545,2019-12-28t21:01:11,lowell ma,pulte pultedaily yes you re do
12,1211029356174282752,2019-12-28t21:01:13,dorchester massachusetts,it s time for lsu football
18,1211029458678882304,2019-12-28t21:01:37,boston ma,met this angel outside of lana del rey s first...
23,1211029479809785857,2019-12-28t21:01:42,boston ma,eliselemassena go elise
...,...,...,...,...
24985,1211694500596375566,2019-12-30t17:04:15,por todo l a califas,@sign0fdoom this is how i meant to sing it to ...
24986,1211694498910064640,2019-12-30t17:04:15,los angeles ca,"with that being said, im going back to sleep w..."
24988,1211694506187223040,2019-12-30t17:04:17,indio ca,mexicans have a fridge in the garage just for ...
24990,1211694506656972806,2019-12-30t17:04:17,los banos ca,"maybe they feel ""unsafe"" cuz you might kick so..."


In [11]:
# average tweet length
np.mean(tweets['text'].str.len())

72.20119867254445

In [16]:
# tweet tokenization and stopword removal
_stopwords = ['https', 'co', 'ct'] + stopwords.words('english')
_keywords = readKeywords()
def processTweet(tweet):
    lem = WordNetLemmatizer()
    tweet_ = tweet.lower() # 1. transform to lowercase
    tweet_ = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet_ = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet_ = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    tweet_ = [word for word in word_tokenize(tweet) if word.isalpha() == True]
    tweet_ = [lem.lemmatize(word,'v') for word in tweet]
    return tweet_

def tokenizeTweet(tweet):
    return [word for word in tweet if word not in _stopwords]

'''
A vocabulary in Natural Language Processing is a list of all speech segments available for the model. In our case, this includes all the words resident in the Training set we have, as the model can make use of all of them relatively equally — at this point, to say the least
'''
def buildVocab(tokenizedData):
    wordlist = nltk.FreqDist(tokenizedData)
    #wordlist.plot(50, cumulative = True)
    word_features = wordlist.keys()
    
    return word_features

def filterKeywords(tweet):
    if any(x in processTweet(tweet) for x in _keywords):
        #print(tweet)
        return True
    #if(tweet in _keywords):
    #    print(tweet)

filtered_tweets = pd.DataFrame(None) # creates an empty dataframe
hits = 0
#for tweet in tweets:
print(tweets.index)
for i in range(len(tweets)):
    tweet = tweets.iloc[i]
    if(filterKeywords(tweet['text']) == True):
        hits += 1
        tweet['text'] = processTweet(tweet['text'])
        filtered_tweets = filtered_tweets.append(tweet)
    #tweet_tokens = processTweet(tweet)
    #for token in tweet_tokens:
    #    processed_tweets.append(token)
print('No. of hits: ', hits)
#print(processed_tweets[0:10])
#print(buildVocab(processed_tweets))
#buildVocab(processed_tweets[:50])


['flu', 'kill', 'awful', 'virus', 'syndrome', 'sickness', 'disease', 'illness', 'ill', 'sick', 'virulent', 'medicine', 'influenza', 'vaccine', 'drug', 'cough', 'cure', 'diagnosis', 'medication', 'sore', 'throat', 'doctor', 'cold', 'stomach', 'ache', 'pain', 'tired', 'nausea']
Int64Index([    0,     1,     2,     3,     5,     6,     7,     9,    10,
               11,
            ...
            24984, 24985, 24986, 24987, 24988, 24989, 24990, 24991, 24992,
            24993],
           dtype='int64', length=20189)
No. of hits:  0


In [17]:
filtered_tweets['loc'].unique()

KeyError: 'loc'

In [15]:
#filtered_tweets[filtered_tweets['loc'].str.contains(' ny')]
filtered_tweets.head(10)

,created,loc,text,tweet_id
153,2019-12-28t21:03:50,worldwide lsu nation,"[heart, heavy, carley, mccord, kill, morning, ...",1.211030e+18
347,2019-12-28t21:07:19,cold spring harbor ny,"[andrewhires, mnitabach, ill, get, home, soon,...",1.211031e+18
382,2019-12-28t21:07:52,commonwealth of massachusetts,"[tfw, sick, cold, remember, doctorwho, maratho...",1.211031e+18
387,2019-12-28t21:07:59,new york,"[johncardillo, turn, stomach]",1.211031e+18
410,2019-12-28t21:08:32,in this bitch,"[hey, friends, wednesday, shave, head, yesterd...",1.211031e+18
816,2019-12-28t21:16:14,east atl,"[bitch, remind, da, time, dat, drink, kill, yo...",1.211033e+18
845,2019-12-28t21:16:50,pueblo co,"[damn, joe, kill, benju, like, pussy]",1.211033e+18
873,2019-12-28t21:17:15,ca on city co,"[king, axis, hp, avix, gb, artvii, nope, im, g...",1.211033e+18
1035,2019-12-28t21:19:56,probably with mollie,"[zachhowell, didnt, picture, bad, ill, fix]",1.211034e+18
1124,2019-12-28t21:21:02,burlington co,"[pdazzleliscious, theceng, actually, live, pai...",1.211034e+18
